# Dependency


## Install dependency
```
sudo apt-get install build-essential gfortran \
     autoconf automake libtool autoconf-archive autotools-dev \
     maxima libblas-dev liblapack-dev \
     libglew-dev libsdl2-dev \
     libfcl-dev libompl-dev \
     sbcl \
     default-jdk \
     blender flex povray ffmpeg \
     coinor-libclp-dev libglpk-dev liblpsolve55-dev libnlopt-dev
sudo apt install z3
```

## Install amino

### get amino
```
git clone https://github.com/golems/amino.git
```

### ompl setting
* Version 1.4.2 of OMPL and the corresponding Debian/Ubuntu packages are missing a required dependency on libeigen. To resolve, you may need to do the following:
  * Manually install libeigen:
  ```
  sudo apt-get install libeigen3-dev
  ```

  * Manually add the eigen include directory to the ompl pkg-config file. The ompl pkg-config file will typically be in /usr/lib/x86_64-linux-gnu/pkgconfig/ompl.pc or /opt/ros/melodic/lib/pkgconfig/ompl.pc. The eigen include path will typically be /usr/include/eigen3. Thus, you may change the Cflags entry in /usr/lib/x86_64-linux-gnu/pkgconfig/ompl.pc and /opt/ros/melodic/lib/pkgconfig/ompl.pc to:
  ```
  Cflags: -I${includedir} -I/usr/include/eigen3
  ```

### Quicklisp Setup 
```
wget https://beta.quicklisp.org/quicklisp.lisp
sbcl --load quicklisp.lisp \
     --eval '(quicklisp-quickstart:install)' \
     --eval '(ql:add-to-init-file)' \
     --eval '(quit)'
```

### build
```
cd amino
git submodule init && git submodule update && autoreconf -i
./configure
make
sudo make install
```

# Install
* make & enter directory for working  
* get repo

```
git clone https://github.com/kavrakilab/tmkit.git
cd tmkit
sudo ldconfig && autoreconf -i && ./configure
make && sudo make install
sudo ldconfig
```

# Test example

## Prepare files

### get baxter description
* go to your worksapce root directory
```
git clone https://github.com/RethinkRobotics/baxter_common
export ROS_PACKAGE_PATH=/home/tamp/Projects/baxter_common # export ROS_PACKAGE_PATH={Your work space}/baxter_common
```

* visualize
```
aarxc --gui package://baxter_description/urdf/baxter.urdf
```

* <font color='red'>**you need to export POS_PACKAGE_PATH everytime to run baxter examples**</font>

### prepare example files

* Copy the example files into a working directory
```
mkdir baxter-blocks
cp tmkit/demo/baxter-sussman/*.robray \
    tmkit/demo/baxter-sussman/q0.tmp \
    tmkit/demo/domains/blocksworld/tm-blocks.* \
    baxter-blocks
```

* visualize
```
cd baxter-blocks
aarxc --gui package://baxter_description/urdf/baxter.urdf sussman-0.robray
```

## test planning

### test default example
* run planner command in baxter-blocks/
```
 tmsmt package://baxter_description/urdf/baxter.urdf \
       sussman-0.robray \
       allowed-collision.robray \
       tm-blocks.pddl \
       tm-blocks.py \
       -q q0.tmp \
       -g sussman-1.robray  \
       -o baxter-sussman.tmp
```

* load and view the computed plan
```
tmsmt package://baxter_description/urdf/baxter.urdf \
        sussman-0.robray \
        allowed-collision.robray \
        baxter-sussman.tmp \
        --gui
```

### changing goal

* Copy and edit the goal scene  
```
cp sussman-1.robray sussman-reversed.robray
gedit sussman-reversed.robray
```

* Change the goal file to reverse stack order  

```
include "class.robray"
include "table.robray"

frame block_c {
    parent block_b;
    translation [0, 0, block_stack];
    geometry {
        isa block;
        isa red;
    }
}

frame block_b {
    parent block_a;
    translation [0, -.25, block_stack];
    geometry {
        isa block;
        isa blue;
    }
}
```

* Re-execute the planner and visualize the plan
```
tmsmt package://baxter_description/urdf/baxter.urdf \
        sussman-0.robray \
        allowed-collision.robray \
        tm-blocks.pddl \
        tm-blocks.py \
        -q q0.tmp \
        -g sussman-reversed.robray  \
        -o baxter-sussman-reversed.tmp \
        --gui
```

### chaning scene

* Copy and edit the start scene  
```
cp sussman-0.robray 4-blocks-start.robray
gedit 4-blocks-start.robray
```

* add additional block  
```
/* the kinematic frame for block "D" */
frame block_d {
    parent block_b; // block_d is initially on block_b
    /* set the height of block_d above block_b */
    translation [0, 0, block_stack];
    geometry {
        isa block;
        color [0,1,1]; // block_d is cyan
    }
}
```
 
* view the new scene  
```
aarxc --gui  package://baxter_description/urdf/baxter.urdf 4-blocks-start.robray
```

* Copy and edit the goal scene
``` 
cp sussman-1.robray 4-blocks-goal.robray
gedit 4-blocks-goal.robray
```

* Add the additional block  
```
frame block_d {
    parent block_a; // block "D" is stacked on block "A"
    /* set the relative height of block "D" */
    translation [0, 0, block_stack];
    geometry {
        isa block;
    }
}
```

* Re-execute the planner and visualize the plan
```
tmsmt package://baxter_description/urdf/baxter.urdf \
        sussman-0.robray \
        allowed-collision.robray \
        tm-blocks.pddl \
        tm-blocks.py \
        -q q0.tmp \
        -g sussman-reversed.robray  \
        -o baxter-sussman-reversed.tmp \
        --gui
```